<a href="https://colab.research.google.com/github/IEncryptSaad/DASHCAM-ANOMALY-CLASSIFIER/blob/main/Dashcam_Anomaly_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**DASHCAM ANOMALY CLASSIFIER**

*(Classify unsafe driving events using image classification and CNNs)*

In [1]:
## STEP NO. 1: Setup & Imports

In [2]:
# Install packages
!pip install -q tensorflow opencv-python-headless

# 📚 Imports
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix


In [4]:
# Install Kaggle CLI
!pip install -q kaggle

# Upload kaggle.json
from google.colab import files
files.upload()  # upload kaggle.json manually here

# Move it to the right location
!mkdir -p ~/.kaggle
!cp kaggle(1).json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle(1).json

# Download the dataset
!kaggle datasets download -d xhlulu/dashcam-accident-detection

# Unzip the dataset
!unzip dashcam-accident-detection.zip -d dashcam_data/


Saving kaggle (1).json to kaggle (1) (1).json
/bin/bash: -c: line 1: syntax error near unexpected token `('
/bin/bash: -c: line 1: `cp kaggle(1).json ~/.kaggle/'
/bin/bash: -c: line 1: syntax error near unexpected token `('
/bin/bash: -c: line 1: `chmod 600 ~/.kaggle/kaggle(1).json'
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 10, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/kaggle/cli.py", line 68, in main
    out = args.func(**command_args)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/kaggle/api/kaggle_api_extended.py", line 1734, in dataset_download_cli
    with self.build_kaggle_client() as kaggle:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/kaggle/api/kaggle_api_extended.py", line 688, in build_kaggle_client
    username=self.config_values['username'],
             ~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^
KeyError: 'username'
unzip